<a href="https://colab.research.google.com/github/rafaxy/tarefas_AI/blob/main/Tarefa_trash_bin_Rafael.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import  Dropout,ZeroPadding2D, MaxPooling2D, Activation, add
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import tensorflow as tf
import cv2
import zipfile
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix

from glob import glob

In [ ]:
from tensorboard.plugins.hparams import api as hp

In [ ]:
!gdown --id 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 105MB/s]


In [ ]:
FILE='trash_nov_22_2018.zip'

In [ ]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [ ]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [ ]:
Img_Size = 80
ref = 'jpg'
notref = 'met'

In [ ]:
X= []
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    #X.append(cv2.imread(name,0))
    img = cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64, 64))
    X.append(img)
m = len(Y)
X = np.array(X)
print(n,m)

2527 2527


In [ ]:
lista =[]
for word in Y:
  lista.append(word[0:4])


In [ ]:
classes = list(set(lista))

In [ ]:
classes

['tras', 'meta', 'pape', 'plas', 'glas', 'card']

In [ ]:
y_data = []
for word in Y:
  if word[0:4]==classes[0]:
    y_data.append(0)
  elif word[0:4]==classes[1]:
    y_data.append(1)
  elif word[0:4]==classes[2]:
    y_data.append(2)
  elif word[0:4]==classes[3]:
    y_data.append(3)
  elif word[0:4]==classes[4]:
    y_data.append(4)
  else:
    y_data.append(5)



In [ ]:
Y = y_data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
df = pd.DataFrame({'class':y_train})
y_train = pd.get_dummies(df['class'])
y_train

,0,1,2,3,4,5
0,0,0,0,0,0,1
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,1,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
2016,0,0,1,0,0,0
2017,0,1,0,0,0,0
2018,0,1,0,0,0,0
2019,0,1,0,0,0,0


In [ ]:
for i in range(6):
  df_plot= df[(df["class"] == i)]
  n= df_plot.shape[0]
  print(classes[i],n)

tras 113
meta 323
pape 476
plas 383
glas 394
card 332


In [ ]:
df = pd.DataFrame({'class':y_test})
y_test = pd.get_dummies(df['class'])
y_test

,0,1,2,3,4,5
0,0,0,1,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
501,0,0,1,0,0,0
502,0,0,0,0,0,1
503,0,0,0,0,1,0
504,0,1,0,0,0,0


In [ ]:
num_classes = 6

In [ ]:
def build_model(hp):
  DROPOUT = hp.Float('dropout', 0, 0.5, step=0.1, default=0.5)
  HP_OPTIMIZER = hp.Choice('OPTIMIZER',['adam', 'sgd','RMSprop'])
  HP_LEARNING_RATE = hp.Choice('LR',[1e-3, 1e-4])
  HP_EPOCHS = hp.Choice('EPOCHS',[100, 200])
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(DROPOUT))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(DROPOUT))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(DROPOUT))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  if 'adam'== HP_OPTIMIZER:
    optimizer = optimizer=tf.keras.optimizers.Adam(learning_rate=HP_LEARNING_RATE )
  elif 'sgd'== HP_OPTIMIZER:
    optimizer = optimizer=tf.keras.optimizers.SGD(learning_rate=HP_LEARNING_RATE )
  else: # 'RMSprop'
    optimizer = optimizer=tf.keras.optimizers.RMSprop(learning_rate=HP_LEARNING_RATE /10)

  #optimizer=hparams[HP_OPTIMIZER]
  #optimizer=hparams[HP_OPTIMIZER](learning_rate = hparams[HP_LEARNING_RATE])
  #optimizer= tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])


  return model

In [ ]:
#!pip install kerastuner
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train,y_train,
             validation_data=(X_test,y_test),
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=5),tf.keras.callbacks.TensorBoard("/tmp/tb_logs")])

Trial 54 Complete [00h 01m 32s]
val_accuracy: 0.5573122501373291

Best val_accuracy So Far: 0.5750988125801086
Total elapsed time: 01h 52m 23s


In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]


In [ ]:
!pip install tensorboard

In [ ]:
import tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /tmp/tb_logs